In [1]:
import re,string

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

 
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import punkt
from nltk.tag import pos_tag
from nltk.corpus import wordnet as wn

  
from string import punctuation
from collections import Counter
 
from collections import OrderedDict
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from afinn import Afinn
from wordcloud import WordCloud, STOPWORDS



In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import sentiwordnet as swn

In [3]:
from nltk.corpus import stopwords

In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import os
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim.models
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt



from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
import csv

#from xgboost import XGBClassifier


In [5]:
#from xgboost import XGBClassifier


In [6]:
porter = PorterStemmer()
wnl = WordNetLemmatizer() 


# In[28]:




In [7]:
#STEMMING = True  # judgment call, parsed documents more readable if False
MAX_NGRAM_LENGTH = 3  # try 1 and 2 and see which yields better modeling results
VECTOR_LENGTH = 100  # set vector length for TF-IDF and Doc2Vec

In [8]:
business = pd.read_csv("/Users/dipenvekaria/Documents/Richa/capstone/business.csv")
#business.head(100)
business.count() #3244 cats, 3247 rest
#business.shape#3247 rows, 22 columns
#business.nunique()#68 cat, review_count 459
#business.isnull().sum()#3 null cats
#yrv


business_id                   3247
name                          3247
address                       3247
city                          3247
state                         3247
postal_code                   3247
latitude                      3247
longitude                     3247
stars                         3247
review_count                  3247
is_open                       3247
categories                    3247
cat                           3244
Fast Food                     3247
Bars                          3247
Gluten-Free                   3247
Vegetarian                    3247
Vegan                         3247
Pizza                         3247
temp                          3247
chain                         3247
ByAppointmentOnly             3221
BusinessAcceptsCreditCards    3221
BusinessParking_garage        3221
BusinessParking_street        3221
BusinessParking_validated     3221
BusinessParking_lot           3221
BusinessParking_valet         3221
RestaurantsPriceRang

In [9]:
tips = pd.read_csv("yelp_tip.csv")
tips.head()



,text,date,likes,business_id,user_id
0,Great breakfast large portions and friendly wa...,2015-08-12,0,jH19V2I9fIslnNhDzPmdkA,ZcLKXikTHYOnYt5VYRO5sg
1,Nice place. Great staff. A fixture in the tow...,2014-06-20,0,dAa0hB2yrnHzVmsCkN4YvQ,oaYhjqBbh18ZhU0bpyzSuw
2,Happy hour 5-7 Monday - Friday,2016-10-12,0,dAa0hB2yrnHzVmsCkN4YvQ,ulQ8Nyj7jCUR8M83SUMoRQ
3,"Parking is a premium, keep circling, you will ...",2017-01-28,0,ESzO3Av0b1_TzKOiqzbQYQ,ulQ8Nyj7jCUR8M83SUMoRQ
4,Homemade pasta is the best in the area,2017-02-25,0,k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ


In [10]:
business_ids = business.business_id.unique().tolist()
len(business_ids) # 3241


3247

In [11]:
reviews = pd.read_csv("yelp_review.csv")
#reviews.count()
#reviews['text'].head(100)
reviews.nunique()

review_id      5261668
user_id        1326101
business_id     174567
stars                5
date              4547
text           5258354
useful             294
funny              255
cool               187
dtype: int64

In [12]:
review = reviews[reviews.business_id.isin(business_ids)]
review.count()

review_id      294858
user_id        294858
business_id    294858
stars          294858
date           294858
text           294858
useful         294858
funny          294858
cool           294858
dtype: int64

In [13]:
tip=tips[tips.business_id.isin(business_ids)]
tip.count()

text           72495
date           72495
likes          72495
business_id    72495
user_id        72495
dtype: int64

In [14]:
tipreview = review.append(tip,ignore_index=True)
tipreview.count()

business_id    367353
cool           294858
date           367353
funny          294858
likes           72495
review_id      294858
stars          294858
text           367353
useful         294858
user_id        367353
dtype: int64

In [15]:
bustipreview = pd.merge(business,tipreview,on='business_id', how = 'left')
bustipreview.count()#367353

business_id                   367353
name                          367353
address                       367353
city                          367353
state                         367353
postal_code                   367353
latitude                      367353
longitude                     367353
stars_x                       367353
review_count                  367353
is_open                       367353
categories                    367353
cat                           367333
Fast Food                     367353
Bars                          367353
Gluten-Free                   367353
Vegetarian                    367353
Vegan                         367353
Pizza                         367353
temp                          367353
chain                         367353
ByAppointmentOnly             367179
BusinessAcceptsCreditCards    367179
BusinessParking_garage        367179
BusinessParking_street        367179
BusinessParking_validated     367179
BusinessParking_lot           367179
B

In [16]:
bustipreview = bustipreview.rename(columns={'name':'business_name','stars_x':'business_star_rating','stars_y':'user_star_rating'})
bustipreview = bustipreview.drop(['cool','funny','likes','useful'],axis=1)

In [43]:
## add column of number of words in review and label of negative and postive reviews
bustipreview['num_words_review'] = bustipreview['text'].str.replace('\n',''). \
                                          str.replace('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','').map(lambda x: len(x.split()))

<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<ipython-input-43-eb416d49c76e>:2: DeprecationWarning: invalid escape sequence \(
  bustipreview['num_words_review'] = bustipreview['text'].str.replace('\n','').                                           str.replace('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','').map(lambda x: len(x.split()))


In [18]:
bustipreview['labels'] =''
bustipreview.loc[bustipreview.user_star_rating>=4.0, 'labels'] = 'positive'
bustipreview.loc[bustipreview.user_star_rating==3.0, 'labels'] = 'neutral'
bustipreview.loc[bustipreview.user_star_rating< 3.0, 'labels'] = 'negative'


In [19]:
#bustipreview.drop(bustipreview[bustipreview['labels'] =='neutral'].index, axis=0, inplace=True)
#bustipreview.reset_index(drop=True, inplace=True)
#bustipreview.head(15)

In [20]:
#bustipreview[bustipreview['labels'] ==''].count()#72495
bustipreview.count()

business_id                   367353
business_name                 367353
address                       367353
city                          367353
state                         367353
postal_code                   367353
latitude                      367353
longitude                     367353
business_star_rating          367353
review_count                  367353
is_open                       367353
categories                    367353
cat                           367333
Fast Food                     367353
Bars                          367353
Gluten-Free                   367353
Vegetarian                    367353
Vegan                         367353
Pizza                         367353
temp                          367353
chain                         367353
ByAppointmentOnly             367179
BusinessAcceptsCreditCards    367179
BusinessParking_garage        367179
BusinessParking_street        367179
BusinessParking_validated     367179
BusinessParking_lot           367179
B

In [21]:
bustipreview.drop(bustipreview[bustipreview['labels'] ==''].index, axis=0, inplace=True)
bustipreview.reset_index(drop=True, inplace=True)
bustipreview.head(15)

,business_id,business_name,address,city,state,postal_code,latitude,longitude,business_star_rating,review_count,...,weekday_dinner,weekend_dinner,weekday_late_night,weekend_late_night,date,review_id,user_star_rating,text,user_id,labels
0,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.51027,-112.028937,3.0,75,...,1,1,0,0,2013-10-02,nGPemoNcDPEZUo2jbD6W1Q,2.0,Skip PB and try True Food instead!\n\nToo bad ...,NwxF1Ftc-4P5Pl_SSXThQA,negative
1,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.51027,-112.028937,3.0,75,...,1,1,0,0,2013-04-23,m9PNVNUxZNhyTXRFCWikPg,3.0,"I love their salads and soups! \n\nYes, Paradi...",EOxdQuedwe_2MkdLnX7MAA,neutral
2,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.51027,-112.028937,3.0,75,...,1,1,0,0,2014-02-06,wE-DaBBGIbVOHGGZ4700pQ,3.0,I always get a thai chicken salad when I come ...,cNkofgeZTGNkmw8m5jR8LA,neutral
3,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.51027,-112.028937,3.0,75,...,1,1,0,0,2014-03-18,aud3jxxba0Z4OIeBjFPYyQ,3.0,When you walk in the door of the Biltmore loca...,LRh7vAnTwvlO0yHP12rxRg,neutral
4,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.51027,-112.028937,3.0,75,...,1,1,0,0,2015-01-14,4HiH3KtKsk5oFacrlFSajw,1.0,"This is one of the few ""affordable"" lunch opti...",pCYGQH-b7YkdR5wT8kvipA,negative
5,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.51027,-112.028937,3.0,75,...,1,1,0,0,2012-02-28,fw_6yyyy01al391t-m1spA,1.0,Won't ever come back here again! I came in and...,NWuFF7-HjYE9dvt_xGbEhw,negative
6,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.51027,-112.028937,3.0,75,...,1,1,0,0,2009-05-28,DtvvqcmagutrFpprdJ-A6w,4.0,I just can't get enough of this place. I know ...,pf7pB-UbvKT-AWfQcg6kBg,positive
7,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.51027,-112.028937,3.0,75,...,1,1,0,0,2014-02-24,jl_smfbQ4BFLxYiguZo_Yg,1.0,For the love of all that is good and right in ...,MD8nhQkHlRtulzPwDOdlcA,negative
8,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.51027,-112.028937,3.0,75,...,1,1,0,0,2015-09-21,unm097CkbAMQmNFMU1UFRA,1.0,"I'll keep it quick, you forgot some of our foo...",mjBN1BCG8ChnNMDTrf7kgQ,negative
9,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016,33.51027,-112.028937,3.0,75,...,1,1,0,0,2015-11-30,MpqCNuVCvK9Q2lhOPJ4Z6Q,4.0,A great office away from home. They cater to t...,vBSzgSvcawxtaDuIdi9hCA,positive


In [22]:
print(bustipreview['labels'].unique())
bustipreview.drop(bustipreview[bustipreview['labels'] =='neutral'].index, axis=0, inplace=True)
bustipreview.reset_index(drop=True, inplace=True)
bustipreview.count()


['negative' 'neutral' 'positive']


business_id                   259578
business_name                 259578
address                       259578
city                          259578
state                         259578
postal_code                   259578
latitude                      259578
longitude                     259578
business_star_rating          259578
review_count                  259578
is_open                       259578
categories                    259578
cat                           259562
Fast Food                     259578
Bars                          259578
Gluten-Free                   259578
Vegetarian                    259578
Vegan                         259578
Pizza                         259578
temp                          259578
chain                         259578
ByAppointmentOnly             259441
BusinessAcceptsCreditCards    259441
BusinessParking_garage        259441
BusinessParking_street        259441
BusinessParking_validated     259441
BusinessParking_lot           259441
B

In [23]:
train_bistiprev = bustipreview.iloc[:181704,] 
test_bistiprev = bustipreview.iloc[181704:,:] 
train_label = bustipreview['labels'][:181704]
test_label = bustipreview['labels'][181704:]
xtrain_bistiprev, xvalid_bistiprev,ytrain, yvalid = train_test_split(train_bistiprev, train_label,random_state=42,test_size=0.3)


In [24]:
#train_bistiprev = bustipreview.iloc[:208940,:] 
print(xtrain_bistiprev['business_id'].count())
print(xvalid_bistiprev['business_id'].count())
print(ytrain.count())
print(yvalid.count())

127192
54512
127192
54512


In [25]:
def vaderize(df, textfield):
    '''Compute the Vader polarity scores for a textfield. 
    Returns scores and original dataframe.'''

    analyzer = SentimentIntensityAnalyzer()

    print('Estimating polarity scores for %d cases.' % len(df))
    sentiment = df[textfield].apply(analyzer.polarity_scores)

    # convert to dataframe
    sdf = pd.DataFrame(sentiment.tolist()).add_prefix('vader_')

    # merge dataframes
    df_combined = pd.concat([df, sdf], axis=1)
    return df_combined

In [26]:
df_vaderized = vaderize(xtrain_bistiprev, 'text')
df_vaderized


Estimating polarity scores for 127192 cases.


,business_id,business_name,address,city,state,postal_code,latitude,longitude,business_star_rating,review_count,...,date,review_id,user_star_rating,text,user_id,labels,vader_compound,vader_neg,vader_neu,vader_pos
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.9280,0.052,0.757,0.191
1,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016.0,33.510270,-112.028937,3.0,75.0,...,2015-01-14,4HiH3KtKsk5oFacrlFSajw,1.0,"This is one of the few ""affordable"" lunch opti...",pCYGQH-b7YkdR5wT8kvipA,negative,0.8442,0.059,0.781,0.161
2,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016.0,33.510270,-112.028937,3.0,75.0,...,2012-02-28,fw_6yyyy01al391t-m1spA,1.0,Won't ever come back here again! I came in and...,NWuFF7-HjYE9dvt_xGbEhw,negative,0.9951,0.019,0.829,0.152
3,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016.0,33.510270,-112.028937,3.0,75.0,...,2009-05-28,DtvvqcmagutrFpprdJ-A6w,4.0,I just can't get enough of this place. I know ...,pf7pB-UbvKT-AWfQcg6kBg,positive,0.9601,0.020,0.796,0.184
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.9803,0.041,0.733,0.227
5,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016.0,33.510270,-112.028937,3.0,75.0,...,2015-09-21,unm097CkbAMQmNFMU1UFRA,1.0,"I'll keep it quick, you forgot some of our foo...",mjBN1BCG8ChnNMDTrf7kgQ,negative,0.9904,0.006,0.789,0.206
6,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016.0,33.510270,-112.028937,3.0,75.0,...,2015-11-30,MpqCNuVCvK9Q2lhOPJ4Z6Q,4.0,A great office away from home. They cater to t...,vBSzgSvcawxtaDuIdi9hCA,positive,0.9187,0.065,0.811,0.124
7,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016.0,33.510270,-112.028937,3.0,75.0,...,2011-06-09,wwHPw3Xw405svqKmigtZYQ,4.0,My boyfriend introduced me to Paradise Bakery ...,9NKn7Pbq23u8yfR08y1BDg,positive,0.9920,0.044,0.750,0.206
8,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016.0,33.510270,-112.028937,3.0,75.0,...,2013-08-04,3aCtGaTvbNK-Xs9Omc99iA,4.0,It would be difficult to find something on the...,K0C1NQhhlMHhRdNmXlyXjw,positive,0.9312,0.000,0.689,0.311
9,__aKnGBedQ51_hEc3D9ARw,"""Paradise Bakery & Cafe""","""2502 E Camelback Rd, Ste 119""",Phoenix,AZ,85016.0,33.510270,-112.028937,3.0,75.0,...,2013-02-06,OKJACNtulpbbOdEkg3Q9tg,1.0,"Finally exposed this ""fresh"" restaurant after ...",ELcQDlf69kb-ihJfxZyL0A,negative,0.9283,0.000,0.791,0.209


In [28]:
xtrain_bistiprev

,business_id,business_name,address,city,state,postal_code,latitude,longitude,business_star_rating,review_count,...,weekday_dinner,weekend_dinner,weekday_late_night,weekend_late_night,date,review_id,user_star_rating,text,user_id,labels
46312,9umA26_aWjEuVe4QW7axtw,"""Paradise Valley Pizza Company""","""4001 East Bell Rd""",Phoenix,AZ,85032,33.639828,-111.995725,4.5,64,...,1,1,0,0,2017-09-07,RdDrEjYBz8UNXGZgIOLF3w,2.0,Small pizza location next to their burger moth...,SRK_sUnvRmMwfexoL_DLQA,negative
179183,pSQFynH1VxkfSmehRXlZWw,"""Pizzeria Bianco""","""623 E Adams St""",Phoenix,AZ,85004,33.449160,-112.065635,4.0,2035,...,1,1,0,0,2014-10-07,4swSkysjw4-0S0aUPmMEhg,1.0,I remember Bianco's before the hype and can't ...,3qAudxedWMgIboNlKlr58w,negative
67550,cOWXtCz7F9GQhPIks8gm6Q,"""SOLO Trattoria""","""2501 E Camelback Rd, Ste 24""",Phoenix,AZ,85016,33.508457,-112.027672,4.5,268,...,1,0,0,0,2016-06-10,YNyXZyXYud6xlSfQ_Ov0OA,5.0,This is an old review from 2013 that just got ...,ROOgm5MX0U9fW-e8UWW1wQ,positive
173121,pHd7QUYFmIt4E4irprwbuw,"""MAD Greens - Phoenix""","""1515 N 7th Ave, Ste 100""",Phoenix,AZ,85007,33.465322,-112.082347,3.5,100,...,0,0,0,0,2016-11-11,KoHkagbo-bijLy81Ze3nSA,4.0,Mad greens really hit the spot today! The Panc...,6ObFF8-uKnOAlXuSH4TlyQ,positive
176250,poSV39UqEg-gpESXafS9-g,"""Angry Crab Shack & BBQ - Ahwatukee""","""3820 E Ray Rd""",Phoenix,AZ,85044,33.317591,-112.003322,4.0,253,...,1,1,0,0,2015-12-22,FeD-OMDIVotLJTmK8pMPvw,4.0,"Hidden in a little strip mall in Tukee, Angry ...",3f7WGTPkj1a8cIWewDFUrg,positive
151666,MpKFIa0lIaOKSuP6SuDNOA,"""Taco & Dilla Parlor""","""1339 E Northern Ave""",Phoenix,AZ,85020,33.552797,-112.053361,4.5,95,...,0,0,0,0,2016-10-23,h5UO2GS4dikNyczMl-LBWw,5.0,"Last night, at my son's school Fall Festival, ...",xnwp2ETfOZMKrG8lu52x4g,positive
149939,MJyKhCf7lu0XPhCbo-TnbQ,"""IHOP""","""4501 E Cactus Rd""",Phoenix,AZ,85032,33.599293,-111.987308,3.0,62,...,0,0,1,1,2014-10-30,-tXW3YlXLzlmeaGNCR0TzA,2.0,"Ordered here today, BIG mistake but won't happ...",5AHmOPUNleZTBIEe4ovomQ,negative
107805,H9tJl_ntsan-YSZElKKK2w,"""3on Smith Cafe""","""3701 E Indian School Rd""",Phoenix,AZ,85018,33.494911,-112.001944,4.5,158,...,0,0,0,0,2017-05-25,5rJpmHIin79m5DxUhznuCw,4.0,"For me, the Smith was impressive from the star...",d_TBs6J3twMy9GChqUEXkg,positive
58406,BA2tYMpTHR3i7X-6qtcWaw,"""Tea Light Cafe""","""7000 E Mayo Blvd, Ste 1084""",Phoenix,AZ,85054,33.658090,-111.932955,4.0,311,...,1,1,0,0,2010-12-18,qOI4zwBESdoLters4VjRqA,5.0,The pork meatball sandwich is the single best ...,uF5V6WYHa38TJU9LFQyjFQ,positive
60868,Bkkwt8E9MHvgCHn4lUFtow,"""Roy's Restaurant""","""5350 E Marriott Dr""",Phoenix,AZ,85054,33.683461,-111.965779,3.5,295,...,0,0,0,0,2012-01-10,vmt0Aa8GjoDVaQ-fiz07oA,4.0,Aloha hour is the best! How can you beat $6 ap...,F04oV9zmhwayogr4D-ljAQ,positive


In [29]:
#df_vaderized2[df_vaderized2['vader_compound'] >0.5]['labels'] =='positive'
#if(df_vaderized2['vader_compound']> 0.5):
#    df_vaderized2['labels']== 'positive'
df_vaderized.loc[(df_vaderized.vader_compound >0.5), 'labels'] = 'positive' 
df_vaderized.loc[(df_vaderized.vader_compound <-0.5), 'labels'] = 'negative' 


In [30]:
#df_vaderized2
xtrain_bistiprev['labels1']=df_vaderized['labels']

/Users/dipenvekaria/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
xtrain_bistiprev

,business_id,business_name,address,city,state,postal_code,latitude,longitude,business_star_rating,review_count,...,weekend_dinner,weekday_late_night,weekend_late_night,date,review_id,user_star_rating,text,user_id,labels,labels1
46312,9umA26_aWjEuVe4QW7axtw,"""Paradise Valley Pizza Company""","""4001 East Bell Rd""",Phoenix,AZ,85032,33.639828,-111.995725,4.5,64,...,1,0,0,2017-09-07,RdDrEjYBz8UNXGZgIOLF3w,2.0,Small pizza location next to their burger moth...,SRK_sUnvRmMwfexoL_DLQA,negative,negative
179183,pSQFynH1VxkfSmehRXlZWw,"""Pizzeria Bianco""","""623 E Adams St""",Phoenix,AZ,85004,33.449160,-112.065635,4.0,2035,...,1,0,0,2014-10-07,4swSkysjw4-0S0aUPmMEhg,1.0,I remember Bianco's before the hype and can't ...,3qAudxedWMgIboNlKlr58w,negative,negative
67550,cOWXtCz7F9GQhPIks8gm6Q,"""SOLO Trattoria""","""2501 E Camelback Rd, Ste 24""",Phoenix,AZ,85016,33.508457,-112.027672,4.5,268,...,0,0,0,2016-06-10,YNyXZyXYud6xlSfQ_Ov0OA,5.0,This is an old review from 2013 that just got ...,ROOgm5MX0U9fW-e8UWW1wQ,positive,negative
173121,pHd7QUYFmIt4E4irprwbuw,"""MAD Greens - Phoenix""","""1515 N 7th Ave, Ste 100""",Phoenix,AZ,85007,33.465322,-112.082347,3.5,100,...,0,0,0,2016-11-11,KoHkagbo-bijLy81Ze3nSA,4.0,Mad greens really hit the spot today! The Panc...,6ObFF8-uKnOAlXuSH4TlyQ,positive,positive
176250,poSV39UqEg-gpESXafS9-g,"""Angry Crab Shack & BBQ - Ahwatukee""","""3820 E Ray Rd""",Phoenix,AZ,85044,33.317591,-112.003322,4.0,253,...,1,0,0,2015-12-22,FeD-OMDIVotLJTmK8pMPvw,4.0,"Hidden in a little strip mall in Tukee, Angry ...",3f7WGTPkj1a8cIWewDFUrg,positive,positive
151666,MpKFIa0lIaOKSuP6SuDNOA,"""Taco & Dilla Parlor""","""1339 E Northern Ave""",Phoenix,AZ,85020,33.552797,-112.053361,4.5,95,...,0,0,0,2016-10-23,h5UO2GS4dikNyczMl-LBWw,5.0,"Last night, at my son's school Fall Festival, ...",xnwp2ETfOZMKrG8lu52x4g,positive,positive
149939,MJyKhCf7lu0XPhCbo-TnbQ,"""IHOP""","""4501 E Cactus Rd""",Phoenix,AZ,85032,33.599293,-111.987308,3.0,62,...,0,1,1,2014-10-30,-tXW3YlXLzlmeaGNCR0TzA,2.0,"Ordered here today, BIG mistake but won't happ...",5AHmOPUNleZTBIEe4ovomQ,negative,negative
107805,H9tJl_ntsan-YSZElKKK2w,"""3on Smith Cafe""","""3701 E Indian School Rd""",Phoenix,AZ,85018,33.494911,-112.001944,4.5,158,...,0,0,0,2017-05-25,5rJpmHIin79m5DxUhznuCw,4.0,"For me, the Smith was impressive from the star...",d_TBs6J3twMy9GChqUEXkg,positive,positive
58406,BA2tYMpTHR3i7X-6qtcWaw,"""Tea Light Cafe""","""7000 E Mayo Blvd, Ste 1084""",Phoenix,AZ,85054,33.658090,-111.932955,4.0,311,...,1,0,0,2010-12-18,qOI4zwBESdoLters4VjRqA,5.0,The pork meatball sandwich is the single best ...,uF5V6WYHa38TJU9LFQyjFQ,positive,positive
60868,Bkkwt8E9MHvgCHn4lUFtow,"""Roy's Restaurant""","""5350 E Marriott Dr""",Phoenix,AZ,85054,33.683461,-111.965779,3.5,295,...,0,0,0,2012-01-10,vmt0Aa8GjoDVaQ-fiz07oA,4.0,Aloha hour is the best! How can you beat $6 ap...,F04oV9zmhwayogr4D-ljAQ,positive,positive


In [32]:
def pos_tag_text(tokens):
    #Convert Penn treebank tag to wordnet because lemmatize uses wordnet tags
    def penn_to_wn_tags(pos_tag):
        if pos_tag.startswith('J'):
            return wn.ADJ
        elif pos_tag.startswith('V'):
            return wn.VERB
        elif pos_tag.startswith('N'):
            return wn.NOUN
        elif pos_tag.startswith('R'):
            return wn.ADV
        else:
            return None
    
    
    #tokens=word_tokenize(text)
    tagged_text = pos_tag(tokens,tagset='universal')
    tagged_tokens = [(word, penn_to_wn_tags(pos_tag)) for word, pos_tag in tagged_text]
    return tagged_tokens

def lemmatize_pos_text(tokens):
    pos_tagged_text=pos_tag_text(tokens)
    lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag else word for word,pos_tag in pos_tagged_text]
#    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_tokens



In [33]:
#english_stop_words = stopwords.words('english')

def clean_text(text): 
    # split document into individual words
    tokens=text.split()
    
    tokens = lemmatize_pos_text(tokens)
    
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
   # tokens = [word for word in tokens if word.isalpha()]
    # # filter out short tokens
    tokens = [word for word in tokens if len(word) > 3]
    # #lowercase all words
    tokens = [word.lower() for word in tokens]
    # # filter out stop words
    
    #stopword_list=english_stop_words+custom_stop_words
    tokens = [w for w in tokens if not w in stopwords]         
    # # word stemming Commented
   # if STEMMING:
    #    ps=PorterStemmer()
     #   tokens=[ps.stem(word) for word in tokens]
    return tokens

In [34]:
#bustipreview1 = bustipreview.groupby(['business_id','labels'])['text'].apply(lambda x: ','.join(x)).reset_index()
#bustipreview1.nunique()
xtrain_bistiprev['text']


46312     Small pizza location next to their burger moth...
179183    I remember Bianco's before the hype and can't ...
67550     This is an old review from 2013 that just got ...
173121    Mad greens really hit the spot today! The Panc...
176250    Hidden in a little strip mall in Tukee, Angry ...
151666    Last night, at my son's school Fall Festival, ...
149939    Ordered here today, BIG mistake but won't happ...
107805    For me, the Smith was impressive from the star...
58406     The pork meatball sandwich is the single best ...
60868     Aloha hour is the best! How can you beat $6 ap...
69746     This is one of my all-time favorite restaurant...
31939     Definitely an odd location, but I love it sinc...
43577     Margherita pizza with baby arugula and prosciu...
95171     It's been a long time since I've been to Z'Tej...
107899    Red Robin is a pretty awesome place to go for ...
66928     I came yesterday with my fiancee and we are re...
164543    Excellent spot for lunch if yo

In [35]:
## filter words
#xtrain_bistiprev.text = xtrain_bistiprev.text.apply(filter_words)
xtrain_bistiprev['labels1'].value_counts()

positive    109944
negative     17248
Name: labels1, dtype: int64

In [36]:

terms_train=list(xtrain_bistiprev['text'])
terms_valid=list(xvalid_bistiprev['text'])
terms_test =list(test_bistiprev['text'])


In [37]:
len(terms_train)

127192

In [76]:
## get bag of words : the frequencies of various words appeared in each review
vectorizer = CountVectorizer()
feature_train_counts=vectorizer.fit_transform(terms_train)
#bow_transformer = CountVectorizer(analyzer=text_process).fit(X)

#feature_train_counts=for text in xtrain_bistiprev['text'] vectorizer.fit_transform(text)
feature_train_counts.shape

(127192, 66545)

In [39]:

feature_train_counts_v=vectorizer.transform(terms_valid)
#bow_transformer = CountVectorizer(analyzer=text_process).fit(X)

#feature_train_counts=for text in xtrain_bistiprev['text'] vectorizer.fit_transform(text)
feature_train_counts_v.shape

#xvalid_bistiprev

(54512, 66545)

In [68]:
feature_train_counts_t=vectorizer.transform(terms_test)
feature_train_counts_t.shape

(77874, 93107)

In [69]:
feature_train_counts_final=vectorizer.transform(bustipreview['text'])

In [41]:
MAX_NGRAM_LENGTH = 2  # try 1 and 2 and see which yields better modeling results
VECTOR_LENGTH = 100  # set vector length for TF-IDF and Doc2Vec

#Note: We stitched the processed text together so the TFIDF vectorizer can work.
#Final section of code has 3 lists used.  2 of which are used for further processing.
#(1) text_body - unused, (2) processed_text (used in Word2Vec), 
#(3) final_processed_text (used in TFIDF), and (4) DSI titles (used in TFIDF Matrix)
 
###############################################################################
### Sklearn TFIDF 
###############################################################################
#note the ngram_range will allow you to include multiple-word tokens within the TFIDF matrix
#Call Tfidf Vectorizer
print('\nWorking on TF-IDF vectorization')
Tfidf=TfidfVectorizer(ngram_range = (1, MAX_NGRAM_LENGTH), 
	max_features = VECTOR_LENGTH)

#fit the vectorizer using final processed documents.  The vectorizer requires the 
#stiched back together document.

TFIDF_matrix=Tfidf.fit_transform(terms_train)     

#creating datafram from TFIDF Matrix
matrix=pd.DataFrame(TFIDF_matrix.toarray(), 
	columns = Tfidf.get_feature_names(), 
	index = xtrain_bistiprev['labels1'])

matrix.to_csv('tfidf-matrix.csv')
print('\nTF-IDF vectorization complete, matrix saved to tfidf-matrix.csv')



Working on TF-IDF vectorization

TF-IDF vectorization complete, matrix saved to tfidf-matrix.csv


In [44]:
TFIDF_matrix_v=Tfidf.transform(terms_valid)     


In [45]:
TFIDF_matrix_t=Tfidf.transform(terms_test) 

In [46]:
###############################################################################
### Explore TFIDF Values
###############################################################################
average_TFIDF={}
for i in matrix.columns:
    average_TFIDF[i]=np.mean(matrix[i])

average_TFIDF_DF = pd.DataFrame(average_TFIDF,
	index = [0]).transpose()

average_TFIDF_DF.columns=['TFIDF']

#calculate Q1 and Q3 range
Q1=np.percentile(average_TFIDF_DF, 25)
Q3=np.percentile(average_TFIDF_DF, 75)
IQR = Q3 - Q1
outlier=Q3+(1.5*IQR)

#words that exceed the Q3+IQR*1.5
outlier_list = average_TFIDF_DF[average_TFIDF_DF['TFIDF'] >= outlier]
print(outlier_list)
print('========')
print(average_TFIDF_DF['TFIDF'])
#can export matrix to csv and explore further if necessary
#enum1 = enumerate(bustipreview1['text'])


        TFIDF
and  0.191639
is   0.103867
it   0.101806
the  0.256492
to   0.124858
was  0.124633
about         0.024238
all           0.036309
also          0.024283
always        0.028593
an            0.025626
and           0.191639
and the       0.039030
are           0.053973
as            0.040456
at            0.042866
back          0.036883
be            0.043684
because       0.019256
been          0.028132
best          0.032997
but           0.061017
by            0.021777
can           0.031388
chicken       0.029058
delicious     0.031479
don           0.021815
even          0.020867
food          0.071346
for           0.083139
for the       0.021193
from          0.029851
get           0.031583
go            0.033266
good          0.061262
got           0.022665
                ...   
some          0.024287
that          0.060594
the           0.256492
the food      0.033465
their         0.036633
there         0.037326
they          0.062237
this          0.070170
this 

In [47]:

#model_w2v = gensim.models.Word2Vec(
 #           xtrain_bistiprev['text'],
  #          size=200, # desired no. of features/independent variables
   #         window=5, # context window size
    #        min_count=2,
     #       sg = 1, # 1 for skip-gram model
      #      hs = 0,
       #     negative = 10, # for negative sampling
        #    workers= 2, # no.of cores
         #   seed = 34) 

#model_w2v.train(xtrain_bistiprev['text'], total_examples= len(xtrain_bistiprev['text']), epochs=20)

In [48]:
#all_words = ' '.join([text for text in bustipreview1['text']]) from wordcloud import WordCloud wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words) 
#plt.figure(figsize=(10, 7)) plt.imshow(wordcloud, interpolation="bilinear") plt.axis('off') plt.show()

In [49]:
#train_bow = feature_train_counts[:4328,:] 
#test_bow = feature_train_counts[4328:,:] 
#train_label = bustipreview1['labels'][:4328]
#test_label = bustipreview1['labels'][4328:]
#xtrain_bow, xvalid_bow,ytrain, yvalid = train_test_split(train_bow, train_label,random_state=42,test_size=0.3)


In [48]:
#print(xtrain_bow.shape)
#print(ytrain.shape)
ytrain.shape
ytrain = xtrain_bistiprev['labels1']
yvalid.nunique()
from scipy import stats


In [77]:
lreg = LogisticRegression() 
# training the model 
lreg.fit(feature_train_counts, ytrain) 
#prediction = lreg.predict_proba(feature_train_counts_v)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [59]:
lreg

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [50]:
#stats.describe(prediction)
print(prediction)
stats.describe(prediction)
np.percentile(prediction, 0.88)

[[0.10206459 0.89793541]
 [0.05463173 0.94536827]
 [0.07877823 0.92122177]
 ...
 [0.087245   0.912755  ]
 [0.22747587 0.77252413]
 [0.1316298  0.8683702 ]]


0.0007915334940464913

In [51]:
prediction_int = prediction[:,1] >= 0.00079 # if prediction is greater than or equal to 0.3 than 1 else 0

In [66]:
#prediction_int = prediction[::,2] >= 0.000009# if prediction is greater than or equal to 0.3 than 1 else 0 
#prediction_int =(prediction[::,2.0] >= 0.000009 )& (prediction[0.0,::] < 0.0000004) & (prediction[:,1.0,:] > 0.000004) 
#stats.describe(prediction[:,1])


In [52]:
prediction_int = prediction_int.astype(np.int) 

In [53]:

prediction_int2=[]
for x,value in np.ndenumerate(prediction_int):
    if value==1: 
        prediction_int2.append("positive")
    else:
        prediction_int2.append("negative")
f1_score(yvalid, prediction_int2, pos_label="positive") # calculating f1 score for the validation set
#np.unique(prediction_int2)

0.8759127098717049

In [54]:
test_pred = lreg.predict_proba(feature_train_counts_t)

In [55]:
test_pred_int = test_pred[:,1] >= 0.00079 
test_pred_int = test_pred_int.astype(np.int)

In [56]:

test_pred_int2=[]
for x,value in np.ndenumerate(test_pred_int):
    if value>0: 
        test_pred_int2.append("positive")
    else: 
        test_pred_int2.append("negative")
test_label1 = test_pred_int2 

#bus_id = bustipreview1['business_id'][4328:]
#btrlreg = pd.DataFrame()
#btrlreg['business_id'] =bus_id
#btrlreg['current_label']=test_label
#btrlreg['predicted_label']=test_label1
#print(btrlreg)
#btrlreg.to_csv('sub_lreg_bow.csv', index=False) # writing data to a CSV file
f1_score(test_label,test_label1, pos_label="positive")

0.8699899126976639

In [74]:
feature_train_counts_final=vectorizer.transform(bustipreview['text'])


In [67]:
feature_train_counts_final.shape

(259578, 93107)

In [78]:
test_pred1 = lreg.predict_proba(feature_train_counts_final)

ValueError: X has 93107 features per sample; expecting 66545

In [ ]:
#gamma = [10,50,100]
##svc = svm.SVC(kernel='linear', C=1, probability=True).fit(feature_train_counts, ytrain) 
#prediction = svc.predict_proba(feature_train_counts_v) 

In [ ]:
prediction_int = prediction[:,1] >= 0.00079 # if prediction is greater than or equal to 0.3 than 1 else 0 
prediction_int = prediction_int.astype(np.int) 

In [ ]:

prediction_int2=[]
for x,value in np.ndenumerate(prediction_int):
    if value>0: 
        prediction_int2.append("positive")
    else: 
        prediction_int2.append("negative")
f1_score(yvalid, prediction_int2, pos_label="positive") # calculating f1 score for the validation set


In [ ]:
test_pred = svc.predict_proba(feature_train_counts_t) 

In [258]:
test_pred_int = test_pred[:,1] >= 0.3 
test_pred_int = test_pred_int.astype(np.int) 
test_pred_int2=[]
for x,value in np.ndenumerate(test_pred_int):
    if value>0: 
        test_pred_int2.append("positive")
    else: 
        test_pred_int2.append("negative")
test_label1 = test_pred_int2 

f1_score(test_label,test_label1, pos_label="positive")

                 business_id current_label predicted_label
4328  gwM0ykDKKpKgKzwtU5djqg      positive        positive
4329  gxEkxkqQrrK62DJnXxXDLg      negative        negative
4330  gxEkxkqQrrK62DJnXxXDLg      positive        positive
4331  h-bBnFFTZ2cNiuBMFu2UaQ      negative        positive
4332  h-bBnFFTZ2cNiuBMFu2UaQ      positive        positive
4333  h-psPOmlY0mCOflb3S3rdQ      negative        negative
4334  h-psPOmlY0mCOflb3S3rdQ      positive        positive
4335  h1j2bxYuFKkTMZcxqmlxGQ      negative        positive
4336  h1j2bxYuFKkTMZcxqmlxGQ      positive        positive
4337  h3T3cdooQ88xPCK3PfSBaQ      negative        negative
4338  h3T3cdooQ88xPCK3PfSBaQ      positive        positive
4339  h4U3h1RbgLvHl0fKSSUhPA      negative        negative
4340  h4U3h1RbgLvHl0fKSSUhPA      positive        positive
4341  h6neYN7OZVt9_ok2bAx6BA      negative        negative
4342  h6neYN7OZVt9_ok2bAx6BA      positive        positive
4343  h7As2jB8bhfFxCMCvdssWA      negative        negati

0.9403973509933775

In [ ]:
rf = RandomForestClassifier(n_estimators=400, random_state=11).fit(feature_train_counts, ytrain) 
#prediction = rf.predict(feature_train_counts_v) 
# validation score 
#f1_score(yvalid, prediction, pos_label="positive")

In [ ]:
final_prediction = rf.predict(feature_train_counts_final) 

In [ ]:
test_pred = rf.predict(feature_train_counts) 

In [285]:
test_label1 = test_pred  
btrrf = pd.DataFrame()
btrrf['business_id'] =bus_id
btrrf['current_label']=test_label
btrrf['predicted_label']=test_label1
print(btrrf)
btrrf.to_csv('sub_svc_bow.csv', index=False) # writing data to a CSV file
f1_score(test_label,test_label1, pos_label="positive")

                 business_id current_label predicted_label
4328  gwM0ykDKKpKgKzwtU5djqg      positive        positive
4329  gxEkxkqQrrK62DJnXxXDLg      negative        negative
4330  gxEkxkqQrrK62DJnXxXDLg      positive        positive
4331  h-bBnFFTZ2cNiuBMFu2UaQ      negative        negative
4332  h-bBnFFTZ2cNiuBMFu2UaQ      positive        positive
4333  h-psPOmlY0mCOflb3S3rdQ      negative        negative
4334  h-psPOmlY0mCOflb3S3rdQ      positive        positive
4335  h1j2bxYuFKkTMZcxqmlxGQ      negative        negative
4336  h1j2bxYuFKkTMZcxqmlxGQ      positive        positive
4337  h3T3cdooQ88xPCK3PfSBaQ      negative        negative
4338  h3T3cdooQ88xPCK3PfSBaQ      positive        positive
4339  h4U3h1RbgLvHl0fKSSUhPA      negative        negative
4340  h4U3h1RbgLvHl0fKSSUhPA      positive        positive
4341  h6neYN7OZVt9_ok2bAx6BA      negative        negative
4342  h6neYN7OZVt9_ok2bAx6BA      positive        positive
4343  h7As2jB8bhfFxCMCvdssWA      negative        negati

0.9775641025641025

In [261]:
lreg.fit(TFIDF_matrix, ytrain) 
prediction = lreg.predict_proba(TFIDF_matrix_v)

In [264]:

prediction_int = prediction[:,1] >= 0.3 
prediction_int = prediction_int.astype(np.int) 
prediction_int2=[]
for x,value in np.ndenumerate(prediction_int):
    if value>0: 
        prediction_int2.append("positive")
    else: 
        prediction_int2.append("negative")
f1_score(yvalid, prediction_int2, pos_label="positive") # calculating f1 score for the validation set


0.9425454545454546

In [ ]:
svc = svm.SVC(kernel='rbf', gamma=50,, C=1, probability=True).fit(xtrain_tfidf, ytrain) 
prediction = svc.predict_proba(xvalid_tfidf) 

In [265]:
prediction_int = prediction[:,1] >= 0.3 
prediction_int = prediction_int.astype(np.int) 
prediction_int2=[]
for x,value in np.ndenumerate(prediction_int):
    if value>0: 
        prediction_int2.append("positive")
    else: 
        prediction_int2.append("negative")
f1_score(yvalid, prediction_int2, pos_label="positive") # calculating f1 score for the validation set


0.9444855662472242

In [267]:
rf = RandomForestClassifier(n_estimators=400, random_state=11).fit(xtrain_tfidf, ytrain) 
prediction = rf.predict(xvalid_tfidf)
f1_score(yvalid, prediction, pos_label="positive")

0.9462528387585163

In [269]:
train_w2v = wordvec_df.iloc[:4328,:] 
test_w2v = wordvec_df.iloc[4328:,:] 
xtrain_w2v = train_w2v.iloc[ytrain.index,:] 
xvalid_w2v = train_w2v.iloc[yvalid.index,:]


In [270]:
lreg.fit(xtrain_w2v, ytrain) 
prediction = lreg.predict_proba(xvalid_w2v) 
prediction_int = prediction[:,1] >= 0.3 
prediction_int = prediction_int.astype(np.int)
prediction_int2=[]
for x,value in np.ndenumerate(prediction_int):
    if value>0: 
        prediction_int2.append("positive")
    else: 
        prediction_int2.append("negative")
f1_score(yvalid, prediction_int2, pos_label="positive") # calculating f1 score for the validation set


0.6795523906408952

In [271]:
svc = svm.SVC(kernel='linear', C=1, probability=True).fit(xtrain_w2v, ytrain) 
prediction = svc.predict_proba(xvalid_w2v) 
prediction_int = prediction[:,1] >= 0.3 
prediction_int = prediction_int.astype(np.int) 
prediction_int2=[]
for x,value in np.ndenumerate(prediction_int):
    if value>0: 
        prediction_int2.append("positive")
    else: 
        prediction_int2.append("negative")
f1_score(yvalid, prediction_int2, pos_label="positive") # calculating f1 score for the validation set


0.6792069140823589

In [272]:
rf = RandomForestClassifier(n_estimators=400, random_state=11).fit(xtrain_w2v, ytrain) 
prediction = rf.predict(xvalid_w2v)
f1_score(yvalid, prediction, pos_label="positive")

0.889060092449923

In [277]:
xgb_model = XGBClassifier(max_depth=6, n_estimators=1000).fit(xtrain_bow, ytrain) 
prediction = xgb_model.predict(xvalid_bow) 
f1_score(yvalid, prediction, pos_label="positive")

0.9629629629629629

In [279]:
test_pred = xgb_model.predict(test_bow) 
test_label1 = test_pred 
btrxgb = pd.DataFrame()
btrxgb['business_id'] =bus_id
btrxgb['current_label']=test_label
btrxgb['predicted_label']=test_label1
print(btrxgb)
btrxgb.to_csv('sub_xgb_bow.csv', index=False) # writing data to a CSV file
f1_score(test_label,test_label1, pos_label="positive")

                 business_id current_label predicted_label
4328  gwM0ykDKKpKgKzwtU5djqg      positive        positive
4329  gxEkxkqQrrK62DJnXxXDLg      negative        negative
4330  gxEkxkqQrrK62DJnXxXDLg      positive        positive
4331  h-bBnFFTZ2cNiuBMFu2UaQ      negative        negative
4332  h-bBnFFTZ2cNiuBMFu2UaQ      positive        positive
4333  h-psPOmlY0mCOflb3S3rdQ      negative        negative
4334  h-psPOmlY0mCOflb3S3rdQ      positive        positive
4335  h1j2bxYuFKkTMZcxqmlxGQ      negative        negative
4336  h1j2bxYuFKkTMZcxqmlxGQ      positive        positive
4337  h3T3cdooQ88xPCK3PfSBaQ      negative        negative
4338  h3T3cdooQ88xPCK3PfSBaQ      positive        positive
4339  h4U3h1RbgLvHl0fKSSUhPA      negative        negative
4340  h4U3h1RbgLvHl0fKSSUhPA      positive        positive
4341  h6neYN7OZVt9_ok2bAx6BA      negative        negative
4342  h6neYN7OZVt9_ok2bAx6BA      positive        positive
4343  h7As2jB8bhfFxCMCvdssWA      negative        negati

0.9586864406779662

In [281]:
xgb = XGBClassifier(max_depth=6, n_estimators=1000).fit(xtrain_tfidf, ytrain) 
prediction = xgb.predict(xvalid_tfidf) 
f1_score(yvalid, prediction, pos_label="positive")

0.9359457422758101

In [282]:
xgb = XGBClassifier(max_depth=6, n_estimators=1000, nthread= 3).fit(xtrain_w2v, ytrain) 
prediction = xgb.predict(xvalid_w2v) 
f1_score(yvalid, prediction, pos_label="positive")

0.9015151515151515

In [375]:
test_pred = rf.predict(feature_train_counts) 
test_label1 = test_pred  
btrrf_final = pd.DataFrame()
bus1_id = bustipreview1['business_id']
btrrf_final['business_id'] =bus1_id
btrrf_final['current_label']=bustipreview1['labels']
btrrf_final['predicted_label']=test_label1
print(btrrf_final)
btrrf_final.to_csv('sub_rf_bow_final.csv', index=False) # writing data to a CSV file
f1_score(bustipreview1['labels'],test_label1, pos_label="negative")

                 business_id current_label predicted_label
0     -050d_XIor1NpCuWkbIVaQ      negative        negative
1     -050d_XIor1NpCuWkbIVaQ      positive        positive
2     -0WegMt6Cy966qlDKhu6jA      negative        negative
3     -0WegMt6Cy966qlDKhu6jA      positive        positive
4     -0aIra_B6iALlfqAriBSYA      negative        negative
5     -0aIra_B6iALlfqAriBSYA      positive        positive
6     -0tgMGl7D9B10YjSN2ujLA      negative        negative
7     -0tgMGl7D9B10YjSN2ujLA      positive        positive
8     -1UMR00eXtwaeh59pEiDjA      negative        negative
9     -1UMR00eXtwaeh59pEiDjA      positive        positive
10    -2isRNVb6PDuBagELL5EBw      negative        negative
11    -2isRNVb6PDuBagELL5EBw      positive        positive
12    -8JaNeG3etLgOVyg2gOIiA      negative        negative
13    -8JaNeG3etLgOVyg2gOIiA      positive        positive
14    -9eNGMp8XiygI8t8QFuFWw      negative        negative
15    -9eNGMp8XiygI8t8QFuFWw      positive        positi

0.9880699460696192

In [356]:
from sklearn.metrics import confusion_matrix, classification_report


In [374]:
#len(rf.feature_importances_)
print(classification_report(bustipreview1['labels'],test_label1))
feature_imp = pd.Series(rf.feature_importances_,index=vectorizer.get_feature_names()).sort_values(ascending=False)
#print(rf.feature_importances_).sort_values(ascending=False)
feature_imp

             precision    recall  f1-score   support

   negative       0.98      0.99      0.99      3044
   positive       0.99      0.98      0.99      3139

avg / total       0.99      0.99      0.99      6183



great            0.009905
delicious        0.009415
worst            0.008222
friendly         0.008107
amazing          0.007414
always           0.006593
rude             0.005708
terrible         0.005669
awesome          0.005585
excellent        0.005542
yummy            0.005539
fresh            0.005347
horrible         0.005173
definitely       0.005173
perfect          0.004554
best             0.004533
love             0.004441
favorite         0.004111
asked            0.004012
tasty            0.003965
fantastic        0.003821
fast             0.003718
loved            0.003545
clean            0.003487
poor             0.003437
yum              0.003432
awful            0.003414
charged          0.003306
disgusting       0.003205
told             0.003148
                   ...   
osobuco          0.000000
oshi             0.000000
osoh             0.000000
osp              0.000000
osq              0.000000
oss              0.000000
ossa             0.000000
ossborn     

In [313]:
#btrrf_final.nunique()
btrrf_final['predicted_label'][0]== 'positive'

False

In [344]:
predicted_int_label = []
for i in btrrf_final.iterrows():
    btrrf_final.loc[btrrf_final.predicted_label == 'positive', 'predicted_int_label'] = 1 
    btrrf_final.loc[btrrf_final.predicted_label == 'negative', 'predicted_int_label'] = 0 

    #  'predicted_label1' == predicted_int_label.append(1)
    #else:
     #   'predicted_label1' == predicted_int_label.append(0)
      #  btrrf_final.loc[btrrf_final.predicted_label == 'positive', 'predicted_int_label'] = 1 

    
#btrrf_final['predicted_busines_rating'] ==    

In [362]:
btrrf_final.shape[0]

6183

In [390]:
test_pred1 = rf.predict(vectorizer.transform([bustipreview1['text'][0]]))
test_pred1
test_pred2 = rf.predict(vectorizer.transform([bustipreview1['text'][1]]))
test_pred2

array(['positive'], dtype=object)

In [380]:
bustipreview1['text'][0]

'I feel like a douche bag for giving Matt\'s only 1 stars when everyone else raves about this place and the owner truly is such a delight so in my defense let me explain:\nThis place is a hot spot, every time I\'ve tried to go there, it\'s packed.  Each time, I\'ve never wanted to wait so I went on my way.\nOne Sunday, I was just down the street from them at Valley Youth Theatre watching Winnie the Pooh with my son and I thought how great it would be to take my son to this popular joint for a bit of breakfast afterward.  When we got out of the theatre, it was pouring rain in giant buckets, one of those rarities in Phoenix where you can\'t see what\'s in front of you cause it\'s raining so hard.  Well, we braved the rain and drove to Matt\'s~ 5 mins. after their posted closing time and yup, you guess it~ despite me and my 3 1\\/2 year old son drenched wet and  looking pleadingly at them for some warm food and a place to stay for awhile to get out of the rain, they told us they were clos